In [1]:
import os
from vis import load_json, compare_solutions, plot_objective_vs_budget

# Define the lambda folders to check
lambda_folders = {
    0.001: "lambda_0001",
    0.01: "lambda_001",
    0.1: "lambda_01",
    1.0: "lambda_1",
    0.9: "lambda_09",
    10.0: "lambda_10",
    100.0: "lambda_100",
    1000.0: "lambda_1000"
}

method = "cmaes"
base_path = f"../results/{method}_hubo_results/"

# Dictionary to store the best results and lambdas for each case
data = {}
best_lambdas = {}

best_exact_data = {}
best_exact_lambdas = {}

# Process each lambda folder
for lambda_val, lambda_folder in lambda_folders.items():
    folder_path = os.path.join(base_path, lambda_folder)
    if not os.path.exists(folder_path):
        print(f"Warning: Folder {folder_path} does not exist. Skipping.")
        continue
        
    files = [file for file in os.listdir(folder_path) if "portfolio_optimization_" in file]
    
    # Process each file
    for file in files:
        file_path = os.path.join(folder_path, file)
        file_data = load_json(file_path)
        
        # For each case in the file
        for case_id, case_data in file_data.items():
            # Get expectation value
            expectation_val = case_data['qaoa_solution']["final_expectation_value"] #['objective_values'][-1]
            
            # If we haven't seen this case before, or if this result is better
            if case_id not in data or expectation_val < data[case_id]['qaoa_solution']["final_expectation_value"]: #['objective_values'][-1]:
                data[case_id] = case_data
                best_lambdas[case_id] = lambda_val

data2 = load_json("../results/classical_unconstrained/filtered_portfolio_optimization_results.json")
data3 = load_json("../results/classical_constrained/filtered_portfolio_optimization_results.json")
data4 = load_json("../results/exact_eigensolver/filtered_portfolio_optimization_results.json")

print(f"Loaded {len(data)} cases.")
nqubits = 15
path = f"./{nqubits}_qubits"
results = compare_solutions(data, data2, data3, data4)

Loaded 100 cases.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def analyze_qaoa_vs_random_probability(results):
    """
    Compare QAOA's most probable state to random sampling probability (1/2^n_qubits)
    
    Parameters:
    -----------
    results : dict
        Dictionary containing case results with QAOA solutions
        
    Returns:
    --------
    pandas.DataFrame: Analysis results by qubit count
    """
    
    analysis_data = []
    
    for case_id, case_data in results.items():
        # Skip if no QAOA solution or states_probs
        if 'qaoa_solution' not in case_data or 'states_probs' not in case_data['qaoa_solution']:
            continue
            
        # Get number of qubits
        #n_qubits = case_data.get('n_qubits', case_data.get('hyperparams', {}).get('n_qubits', 0))
        #if n_qubits == 0:
        #    continue

        n_qubits = case_data["n_qubits"]
            
        # Get the states probabilities
        states_probs = case_data['qaoa_solution']['states_probs']
        
        # Find the maximum probability
        max_prob = max(states_probs)
        
        # Calculate random probability
        random_prob = 1 / (2 ** n_qubits)
        
        # Calculate enhancement factor
        enhancement_factor = max_prob / random_prob
        
        analysis_data.append({
            'case_id': case_id,
            'n_qubits': n_qubits,
            'max_prob': max_prob,
            'random_prob': random_prob,
            'enhancement_factor': enhancement_factor,
            'best_lambda': best_lambdas.get(case_id, 'unknown')
        })
    
    return pd.DataFrame(analysis_data)

def plot_qaoa_enhancement(df):
    """
    Create visualizations comparing QAOA to random sampling
    """
    
    # Group by qubit count
    qubit_stats = df.groupby('n_qubits').agg({
        'enhancement_factor': ['mean', 'std', 'min', 'max', 'count'],
        'max_prob': ['mean', 'std']
    }).reset_index()
    
    # Flatten column names
    qubit_stats.columns = ['n_qubits', 'mean_enhancement', 'std_enhancement', 
                          'min_enhancement', 'max_enhancement', 'count',
                          'mean_max_prob', 'std_max_prob']
    
    # Create comprehensive plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Enhancement factor by qubit count
    ax1.bar(qubit_stats['n_qubits'], qubit_stats['mean_enhancement'], 
            yerr=qubit_stats['std_enhancement'], capsize=5,
            color='steelblue', edgecolor='black', alpha=0.7)
    
    # Add horizontal line at y=1 (no enhancement)
    ax1.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Random level')
    
    ax1.set_title('QAOA Enhancement over Random Sampling', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Number of Qubits', fontsize=12)
    ax1.set_ylabel('Enhancement Factor (QAOA/Random)', fontsize=12)
    ax1.set_yscale('log')  # Log scale since enhancement decreases exponentially
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add count annotations
    for i, row in qubit_stats.iterrows():
        ax1.annotate(f"n={int(row['count'])}", 
                    (row['n_qubits'], row['mean_enhancement']),
                    ha='center', va='bottom', fontsize=9)
    
    # 2. Individual data points with jitter
    for n_qubits in df['n_qubits'].unique():
        qubit_data = df[df['n_qubits'] == n_qubits]
        x_jitter = np.random.normal(0, 0.1, size=len(qubit_data))
        ax2.scatter(qubit_data['n_qubits'] + x_jitter, qubit_data['enhancement_factor'],
                   alpha=0.6, s=50, edgecolors='black')
    
    # Add mean line
    ax2.plot(qubit_stats['n_qubits'], qubit_stats['mean_enhancement'], 
             'r-o', linewidth=2, markersize=8, label='Mean enhancement')
    ax2.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Random level')
    
    ax2.set_title('Enhancement Factor Distribution', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Number of Qubits', fontsize=12)
    ax2.set_ylabel('Enhancement Factor (log scale)', fontsize=12)
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. Maximum probability comparison
    ax3.bar(qubit_stats['n_qubits'] - 0.2, qubit_stats['mean_max_prob'], 
            width=0.4, label='QAOA Max Prob', color='blue', alpha=0.7)
    
    # Calculate and plot random probabilities
    random_probs = [1/(2**q) for q in qubit_stats['n_qubits']]
    ax3.bar(qubit_stats['n_qubits'] + 0.2, random_probs, 
            width=0.4, label='Random Prob', color='red', alpha=0.7)
    
    ax3.set_title('Probability Comparison: QAOA vs Random', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Number of Qubits', fontsize=12)
    ax3.set_ylabel('Probability (log scale)', fontsize=12)
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # 4. Summary statistics table as text
    ax4.axis('off')
    
    # Create summary table
    summary_text = "Summary Statistics by Qubit Count:\n\n"
    summary_text += f"{'Qubits':<8} {'Mean Enh.':<12} {'Max Enh.':<12} {'Cases':<8} {'Random Prob':<15}\n"
    summary_text += "-" * 65 + "\n"
    
    for _, row in qubit_stats.iterrows():
        random_prob = 1/(2**row['n_qubits'])
        summary_text += f"{int(row['n_qubits']):<8} {row['mean_enhancement']:<12.2f} {row['max_enhancement']:<12.2f} {int(row['count']):<8} {random_prob:<15.2e}\n"
    
    ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes, 
            fontsize=11, fontfamily='monospace', va='top')
    
    plt.tight_layout()
    plt.savefig(f'{nqubits}_qubits_qaoa_enhancement_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return qubit_stats

# Run the analysis
print("Analyzing QAOA probability enhancement...")
analysis_df = analyze_qaoa_vs_random_probability(results)

print(f"\nAnalyzed {len(analysis_df)} cases across {analysis_df['n_qubits'].nunique()} different qubit counts.")

# Generate plots and statistics
qubit_summary = plot_qaoa_enhancement(analysis_df)

# Print detailed results
print("\nDetailed Enhancement Analysis:")
print("="*60)
for n_qubits in sorted(analysis_df['n_qubits'].unique()):
    qubit_data = analysis_df[analysis_df['n_qubits'] == n_qubits]
    random_prob = 1/(2**n_qubits)
    
    print(f"\n{n_qubits} Qubits (Random prob: {random_prob:.2e}):")
    print(f"  Cases analyzed: {len(qubit_data)}")
    print(f"  Mean enhancement: {qubit_data['enhancement_factor'].mean():.2f}x")
    print(f"  Best enhancement: {qubit_data['enhancement_factor'].max():.2f}x")
    print(f"  Worst enhancement: {qubit_data['enhancement_factor'].min():.2f}x")
    print(f"  Mean max probability: {qubit_data['max_prob'].mean():.4f}")
    
    # Show best case
    best_case = qubit_data.loc[qubit_data['enhancement_factor'].idxmax()]
    print(f"  Best case: {best_case['case_id']} (λ={best_case['best_lambda']}, {best_case['enhancement_factor']:.2f}x enhancement)")

# Overall summary
print(f"\nOverall Summary:")
print(f"Average enhancement across all cases: {analysis_df['enhancement_factor'].mean():.2f}x")
print(f"Maximum enhancement observed: {analysis_df['enhancement_factor'].max():.2f}x")
print(f"Cases with >2x enhancement: {len(analysis_df[analysis_df['enhancement_factor'] > 2])}/{len(analysis_df)} ({len(analysis_df[analysis_df['enhancement_factor'] > 2])/len(analysis_df)*100:.1f}%)")

Analyzing QAOA probability enhancement...


KeyError: 'n_qubits'